In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = ""

In [2]:
agent_instructions = """You are a helpful assistant. Help the user answer any questions.

You have access to the following tools:

{tools}

In order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. \
You will then get back a response in the form <observation></observation>
For example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respond:

<tool>search</tool><tool_input>weather in SF</tool_input>
<observation>64 degrees</observation>

When you are done, respond with a final answer between <final_answer></final_answer>. For example:

<final_answer>The weather in SF is 64 degrees</final_answer>

Begin!

Question: {question}"""

In [3]:
from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate, AIMessagePromptTemplate
from langchain.agents import tool

/Users/harrisonchase/.pyenv/versions/3.9.1/envs/langchain/lib/python3.9/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.14) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [4]:
model = ChatAnthropic(model="claude-2")

In [5]:
prompt_template = ChatPromptTemplate.from_template(agent_instructions) + AIMessagePromptTemplate.from_template("{intermediate_steps}")

In [6]:
chain = prompt_template | model.bind(stop=["</tool_input>", "</final_answer>"])

In [7]:
@tool
def search(query: str) -> str:
    """Search things about current events."""
    return "32 degrees"

In [8]:
tool_list = [search]

In [9]:
from langchain.agents import Tool, AgentExecutor, BaseSingleActionAgent
from typing import List, Tuple, Any, Union
from langchain.schema import AgentAction, AgentFinish


class AnthropicAgent(BaseSingleActionAgent):
    
    tools: List[Tool]
    chain: Any

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        log = ""
        for action, observation in intermediate_steps:
            log += f"<tool>{action.tool}</tool><tool_input>{action.tool_input}</tool_input><observation>{observation}</observation>"
        tools = ""
        for tool in self.tools:
            tools += f"{tool.name}: {tool.description}\n"
        response = self.chain.invoke({"intermediate_steps": log, "tools": tools, "question": kwargs["input"]})
        if "</tool>" in response.content:
            t, ti = response.content.split("</tool>")
            _t = t.split("<tool>")[1]
            _ti = ti.split("<tool_input>")[1]
            return AgentAction(tool=_t, tool_input=_ti, log=response.content)
        elif "<final_answer>" in response.content:
            t, ti = response.content.split("<final_answer>")
            return AgentFinish(return_values={"output": ti}, log=response.content)
        else:
            raise ValueError

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        raise ValueError

In [10]:
agent = AnthropicAgent(tools=tool_list, chain=chain)

In [11]:
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)

In [12]:
agent_executor.run("whats the weather in New york?")



> Entering new AgentExecutor chain...
 <tool>search</tool>
<tool_input>weather in new york32 degrees

<final_answer>The weather in New York is 32 degrees

> Finished chain.


'The weather in New York is 32 degrees'